In [199]:
import os
import sys
paths = {
    'helpers': '/home/spadela/mids-w210-capstone'
}
for k,v in paths.items():
    sys.path.insert(0, v)
    
import requests
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

from bs4 import BeautifulSoup

### Set Up Mongo Connection
from db import dbToolsMongo
db = dbToolsMongo()

import time
import urllib.parse

pd.set_option('display.max_columns', 500)

%reload_ext autoreload
%autoreload 2

In [200]:
# Delete all records with price = 0 or source = null
db.db.sp_price_history.delete_many({"price": {"$lte": 0.0}})
db.db.sp_price_history.delete_many({"source": None})

df1 = db.toDF('sp_price_history')
df1['close_date'] = df1['close_date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))
df1 = df1.drop_duplicates(subset=['name','close_date'])
df1.sort_values(by=['name','close_date']).head()

,_id,close_date,name,price,source,symbol
6148,5b4278f057c41009c0b628e8,2018-05-24,AB-CHAIN,0.05845,CryptoCompare,RTB
6149,5b4278f057c41009c0b628e9,2018-05-25,AB-CHAIN,0.07280,CryptoCompare,RTB
6150,5b4278f057c41009c0b628ea,2018-05-26,AB-CHAIN,0.07287,CryptoCompare,RTB
6151,5b4278f057c41009c0b628eb,2018-05-27,AB-CHAIN,0.07206,CryptoCompare,RTB
6152,5b4278f057c41009c0b628ec,2018-05-28,AB-CHAIN,0.05735,CryptoCompare,RTB


In [201]:
x = db.toDF('sp_price_history',filters={'name': 'EOS'}).sort_values(by='close_date')
x.head()

,_id,close_date,name,price,source,symbol
0,5b42789157c41009c0b610d5,2017-06-29,EOS,0.850,CryptoCompare,EOS
1,5b42789157c41009c0b610d6,2017-06-30,EOS,0.912,CryptoCompare,EOS
746,5b4a3be057c410184dd6f1d1,2017-07-01,EOS,1.010,CoinMarketCap,EOS
2,5b42789157c41009c0b610d7,2017-07-01,EOS,1.050,CryptoCompare,EOS
3,5b42789157c41009c0b610d8,2017-07-02,EOS,3.000,CryptoCompare,EOS


In [206]:
def compute_key_dates(df,intervals):
    key_dates = df[['name','close_date']].groupby(['name']).agg('min').reset_index().fillna(datetime.datetime(2099,12,31,0,0))
    for i,d in enumerate(intervals):
        if i == 0:
            key_dates['d0'] = key_dates['close_date']
        else:
            key_dates['d'+str(d)] = key_dates['d0'].apply(lambda x: x + datetime.timedelta(days=int(d)))
            key_dates['d'+str(d)] = key_dates['d'+str(d)].apply(lambda x: datetime.datetime(2018,7,8,0,0) if x > datetime.datetime(2018,7,8,0,0) else x)
        
    return key_dates.drop(columns=['close_date'])

In [219]:
df2 = df1[['name']].drop_duplicates()
intervals = [0,7,14,30,90,180]
key_dates = compute_key_dates(df1,intervals)
df2 = pd.merge(df2,key_dates,how='left').drop_duplicates()
# add prices
for d in intervals:
    df2 = pd.merge(df2,df1,left_on=['name','d'+str(d)],right_on=['name','close_date'],how='left').fillna(0)
    df2['p'+str(d)] = df2['price']
    if d > 0:
        df2['pch'+str(d)] = df2.apply(lambda x: np.round(x['p'+str(d)]*1.0/x['p0']-1,2),axis=1)
        df2['y'+str(d)] = df2.apply(lambda x: 1 if x['pch'+str(d)] <= -0.5 else 0,axis=1)
    df2 = df2.drop(columns=['price','_id','source','close_date','symbol'])
df2.head()

,name,d0,d7,d14,d30,d90,d180,p0,p7,pch7,y7,p14,pch14,y14,p30,pch30,y30,p90,pch90,y90,p180,pch180,y180
0,PATRON,2018-05-22,2018-05-29,2018-06-05,2018-06-21,2018-07-08,2018-07-08,0.068146,0.083290,0.22,0,0.052920,-0.22,0,0.039430,-0.42,0,0.027960,-0.59,1,0.027960,-0.59,1
1,Sentinel Protocol,2018-07-05,2018-07-08,2018-07-08,2018-07-08,2018-07-08,2018-07-08,0.057510,0.062690,0.09,0,0.062690,0.09,0,0.062690,0.09,0,0.062690,0.09,0,0.062690,0.09,0
2,Blockchain Traded Fund,2018-06-12,2018-06-19,2018-06-26,2018-07-08,2018-07-08,2018-07-08,33.590000,3.050000,-0.91,1,1.450000,-0.96,1,2.560000,-0.92,1,2.560000,-0.92,1,2.560000,-0.92,1
3,EOS,2017-06-29,2017-07-06,2017-07-13,2017-07-29,2017-09-27,2017-12-26,0.850000,3.230000,2.80,0,1.710000,1.01,0,1.810000,1.13,0,0.631400,-0.26,0,8.600000,9.12,0
4,Fitrova,2018-05-11,2018-05-18,2018-05-25,2018-06-10,2018-07-08,2018-07-08,0.004725,0.008666,0.83,0,0.002871,-0.39,0,0.000115,-0.98,1,0.000015,-1.00,1,0.000015,-1.00,1


In [221]:
db.db['sp_price_features'].delete_many({})
db.db['sp_price_features'].insert_many(df2.to_dict('records'))

In [225]:
db.toDF('sp_price_features')[['name','d0','p0','d7','p7','pch7','y7','d30','p30','pch30','y30']].head()

,name,d0,p0,d7,p7,pch7,y7,d30,p30,pch30,y30
0,PATRON,2018-05-22,0.068146,2018-05-29,0.083290,0.22,0,2018-06-21,0.039430,-0.42,0
1,Sentinel Protocol,2018-07-05,0.057510,2018-07-08,0.062690,0.09,0,2018-07-08,0.062690,0.09,0
2,Blockchain Traded Fund,2018-06-12,33.590000,2018-06-19,3.050000,-0.91,1,2018-07-08,2.560000,-0.92,1
3,EOS,2017-06-29,0.850000,2017-07-06,3.230000,2.80,0,2017-07-29,1.810000,1.13,0
4,Fitrova,2018-05-11,0.004725,2018-05-18,0.008666,0.83,0,2018-06-10,0.000115,-0.98,1
